In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://10.0.0.165:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/mschachter/code/bioinformatics/hail-20210302-0428-0.2.62-84fa81b9ea3d.log


In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [4]:
import os
data_dir = '/home/mschachter/data/simons'
vcf_file = os.path.join(data_dir, 'LP6005443-DNA_C07.annotated.nh2.variants.vcf')

In [4]:
mt_file = os.path.join(data_dir, 'LP6005443-DNA_C07.annotated.nh2.variants.mt')
#hl.import_vcf(vcf_file).write(mt_file, overwrite=True)

In [5]:
mt = hl.read_matrix_table(mt_file)

In [6]:
mt.rows().select().show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:10231,"[""C"",""A""]"
1:10250,"[""A"",""C""]"
1:10321,"[""C"",""T""]"
1:10583,"[""G"",""A""]"
1:12783,"[""G"",""A""]"


In [7]:
mt.entry.show(5)

+---------------+------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+
| locus         | alleles    | 'LP6005443-DNA_C07'.AD | 'LP6005443-DNA_C07'.DP | 'LP6005443-DNA_C07'.GQ | 'LP6005443-DNA_C07'.GT | 'LP6005443-DNA_C07'.PL | 'LP6005443-DNA_C07'.FL |
+---------------+------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+
| locus<GRCh37> | array<str> | array<int32>           |                  int32 |                  int32 | call                   | array<int32>           | str                    |
+---------------+------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+
| 1:10231       | ["C","A"]  | [315,30]               |                    333 |                     78 | 0/1                    | [78,0,576]             | "N"                    |
| 1:10250       | ["A","C"]  | [181,62]               |                    234 |                     36 | 0/1                    | [36,0,1054]            | "N"                    |
| 1:10321       | ["C","T"]  | [212,27]               |                    232 |                     28 | 0/1                    | [28,0,858]             | "N"                    |
| 1:10583       | ["G","A"]  | [17,10]                |                     27 |                     99 | 0/1                    | [120,0,346]            | "N"                    |
| 1:12783       | ["G","A"]  | [14,54]                |                     66 |                     39 | 1/1                    | [385,39,0]             | "N"                    |
+---------------+------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+
showing top 5 rows

# Basic Count and Quality Stats

In [8]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

[(Struct(ref='C', alt='T'), 676288),
 (Struct(ref='G', alt='A'), 673730),
 (Struct(ref='A', alt='G'), 646474),
 (Struct(ref='T', alt='C'), 646462),
 (Struct(ref='G', alt='T'), 176199),
 (Struct(ref='C', alt='A'), 175848),
 (Struct(ref='C', alt='G'), 172103),
 (Struct(ref='G', alt='C'), 171731),
 (Struct(ref='A', alt='C'), 171702),
 (Struct(ref='T', alt='G'), 171375),
 (Struct(ref='A', alt='T'), 157704),
 (Struct(ref='T', alt='A'), 157189)]

In [9]:
mt.rows().show(5)

+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| locus         | alleles    | rsid          |     qual | filters  | info.AC      | info.AF        | info.AN | info.BaseCounts | info.BaseQRankSum | info.DB | info.DP | info.DS | info.Dels |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| locus<GRCh37> | array<str> | str           |  float64 | set<str> | array<int32> | array<float64> |   int32 | array<int32>    |           float64 |    bool |   int32 |    bool |   float64 |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| 1:10231       | ["C","A"]  | "rs200279319" | 7.80e+01 | NA       | [1]          | [5.00e-01]     |       2 | [30,315,2,3]    |         -6.56e-01 |    true |     395 |   false |  1.10e-01 |
| 1:10250       | ["A","C"]  | "rs199706086" | 3.60e+01 | NA       | [1]          | [5.00e-01]     |       2 | [181,62,1,2]    |          2.05e+00 |    true |     255 |   false |  4.00e-02 |
| 1:10321       | ["C","T"]  | NA            | 2.80e+01 | NA       | [1]          | [5.00e-01]     |       2 | [3,212,2,27]    |          6.20e-01 |   false |     244 |   false |  0.00e+00 |
| 1:10583       | ["G","A"]  | "rs58108140"  | 1.20e+02 | NA       | [1]          | [5.00e-01]     |       2 | [10,0,17,1]     |         -2.06e+00 |    true |      28 |   false |  0.00e+00 |
| 1:12783       | ["G","A"]  | NA            | 3.85e+02 | NA       | [2]          | [1.00e+00]     |       2 | [54,1,14,0]     |                NA |   false |      69 |   false |  0.00e+00 |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+

+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
|  info.FS |  info.GC | info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |  info.QD | info.RPA     | info.RU | info.ReadPosRankSum |
+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
|  float64 |  float64 |             float64 |              float64 | array<int32> | array<float64> |  float64 |    int32 |        float64 |  float64 | array<int32> | str     |             float64 |
+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
| 4.93e+00 | 5.19e+01 |            1.82e+02 |                   NA | [1]          | [5.00e-01]     | 1.44e+01 |      213 |       2.93e+00 | 2.00e-01 | NA           | NA      |            3.80e+00 |
| 0.00e+00 | 5.21e+01 |            1.47e+02 |                   NA | [1]          | [5.00e-01]     | 1.72e+01 |      115 |      -1.74e+00 | 1.40e-01 | NA           | NA      |           -4.22e-01 |
| 0.00e+00 | 5.51e+01 |            8.49e+01 |                   NA | [1]          | [5.00e-01]     | 2.09e+01 |       79 |       4.57e-01 | 1.10e-01 | NA           | NA      |           -1.92e+00 |
| 0.00e+00 | 7.11e+01 |            3.25e+00 |                   NA | [1]          | [5.00e-01]     | 4.02e+01 |        0 |       1.20e+00 | 4.29e+00 | NA           | NA      |           -1.81e-01 |
| 0.00e+00 | 6.09e+01 |            1.78e+00 |                   NA | [2]          | [1.00e+00]     | 1.28e+01 |     

In [10]:
p = hl.plot.histogram(mt.DP, range=(0,90), bins=30, title='DP: Filtered Depth', legend='DP')
show(p)

In [11]:
p = hl.plot.histogram(mt.GQ, range=(0, 100), bins=30, title='GQ: Genotype Quality', legend='GQ')
show(p)

In [12]:
p = hl.plot.histogram(mt.info.MQ, range=(0, 60), bins=30, title='MQ: Mapping Quality', legend='MQ')
show(p)

In [13]:
p = hl.plot.histogram(mt.info.FS, range=(0, 30), bins=30, title='FS: Strand Bias Fisher Score', legend='FS')
show(p)

In [14]:
p = hl.plot.histogram(mt.info.HaplotypeScore, range=(0, 30), bins=30, title='HaplotypeScore', legend='HaplotypeScore')
show(p)

# Filtering

In [15]:
tbl = mt.rows()
num_variants_start = tbl.count()
print('# of variants: ', num_variants_start)

# of variants:  3996805


In [16]:
tbl.show(5)

+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| locus         | alleles    | rsid          |     qual | filters  | info.AC      | info.AF        | info.AN | info.BaseCounts | info.BaseQRankSum | info.DB | info.DP | info.DS | info.Dels |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| locus<GRCh37> | array<str> | str           |  float64 | set<str> | array<int32> | array<float64> |   int32 | array<int32>    |           float64 |    bool |   int32 |    bool |   float64 |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+
| 1:10231       | ["C","A"]  | "rs200279319" | 7.80e+01 | NA       | [1]          | [5.00e-01]     |       2 | [30,315,2,3]    |         -6.56e-01 |    true |     395 |   false |  1.10e-01 |
| 1:10250       | ["A","C"]  | "rs199706086" | 3.60e+01 | NA       | [1]          | [5.00e-01]     |       2 | [181,62,1,2]    |          2.05e+00 |    true |     255 |   false |  4.00e-02 |
| 1:10321       | ["C","T"]  | NA            | 2.80e+01 | NA       | [1]          | [5.00e-01]     |       2 | [3,212,2,27]    |          6.20e-01 |   false |     244 |   false |  0.00e+00 |
| 1:10583       | ["G","A"]  | "rs58108140"  | 1.20e+02 | NA       | [1]          | [5.00e-01]     |       2 | [10,0,17,1]     |         -2.06e+00 |    true |      28 |   false |  0.00e+00 |
| 1:12783       | ["G","A"]  | NA            | 3.85e+02 | NA       | [2]          | [1.00e+00]     |       2 | [54,1,14,0]     |                NA |   false |      69 |   false |  0.00e+00 |
+---------------+------------+---------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+

+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
|  info.FS |  info.GC | info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |  info.QD | info.RPA     | info.RU | info.ReadPosRankSum |
+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
|  float64 |  float64 |             float64 |              float64 | array<int32> | array<float64> |  float64 |    int32 |        float64 |  float64 | array<int32> | str     |             float64 |
+----------+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+
| 4.93e+00 | 5.19e+01 |            1.82e+02 |                   NA | [1]          | [5.00e-01]     | 1.44e+01 |      213 |       2.93e+00 | 2.00e-01 | NA           | NA      |            3.80e+00 |
| 0.00e+00 | 5.21e+01 |            1.47e+02 |                   NA | [1]          | [5.00e-01]     | 1.72e+01 |      115 |      -1.74e+00 | 1.40e-01 | NA           | NA      |           -4.22e-01 |
| 0.00e+00 | 5.51e+01 |            8.49e+01 |                   NA | [1]          | [5.00e-01]     | 2.09e+01 |       79 |       4.57e-01 | 1.10e-01 | NA           | NA      |           -1.92e+00 |
| 0.00e+00 | 7.11e+01 |            3.25e+00 |                   NA | [1]          | [5.00e-01]     | 4.02e+01 |        0 |       1.20e+00 | 4.29e+00 | NA           | NA      |           -1.81e-01 |
| 0.00e+00 | 6.09e+01 |            1.78e+00 |                   NA | [2]          | [1.00e+00]     | 1.28e+01 |     

In [17]:
tbl = tbl.filter((tbl.info.DP >= 10) & (tbl.info.DP <= 60))
print('# of variants: {} / {}'.format(tbl.count(), num_variants_start))

# of variants: 3845179 / 3996805


In [18]:
tbl = tbl.filter((tbl.info.MQ >= 55))
print('# of variants: {} / {}'.format(tbl.count(), num_variants_start))

# of variants: 3389861 / 3996805


In [19]:
tbl = tbl.filter((tbl.info.HaplotypeScore < 10))
print('# of variants: {} / {}'.format(tbl.count(), num_variants_start))

# of variants: 3254248 / 3996805


In [20]:
tbl = tbl.filter((tbl.rsid != 'NA'))
print('# of variants: {} / {}'.format(tbl.count(), num_variants_start))

# of variants: 3116330 / 3996805


In [21]:
tbl.show(5)

+---------------+------------+--------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+----------+
| locus         | alleles    | rsid         |     qual | filters  | info.AC      | info.AF        | info.AN | info.BaseCounts | info.BaseQRankSum | info.DB | info.DP | info.DS | info.Dels |  info.FS |
+---------------+------------+--------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+----------+
| locus<GRCh37> | array<str> | str          |  float64 | set<str> | array<int32> | array<float64> |   int32 | array<int32>    |           float64 |    bool |   int32 |    bool |   float64 |  float64 |
+---------------+------------+--------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+----------+
| 1:713977      | ["C","T"]  | "rs74512038" | 9.55e+02 | NA       | [2]          | [1.00e+00]     |       2 | [0,1,0,33]      |          1.39e+00 |    true |      34 |   false |  0.00e+00 | 0.00e+00 |
| 1:726481      | ["T","G"]  | "rs3131980"  | 1.84e+03 | NA       | [2]          | [1.00e+00]     |       2 | [0,0,52,0]      |                NA |    true |      52 |   false |  0.00e+00 | 0.00e+00 |
| 1:752566      | ["G","A"]  | "rs3094315"  | 1.62e+03 | NA       | [2]          | [1.00e+00]     |       2 | [45,0,0,0]      |                NA |    true |      45 |   false |  0.00e+00 | 0.00e+00 |
| 1:755274      | ["C","T"]  | "rs78408995" | 1.16e+03 | NA       | [2]          | [1.00e+00]     |       2 | [0,0,0,39]      |                NA |    true |      39 |   false |  0.00e+00 | 0.00e+00 |
| 1:755775      | ["A","G"]  | "rs3131965"  | 9.17e+02 | NA       | [2]          | [1.00e+00]     |       2 | [1,0,34,0]      |                NA |    true |      35 |   false |  0.00e+00 | 0.00e+00 |
+---------------+------------+--------------+----------+----------+--------------+----------------+---------+-----------------+-------------------+---------+---------+---------+-----------+----------+

+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+----------+
|  info.GC | info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |  info.QD | info.RPA     | info.RU | info.ReadPosRankSum | info.STR |
+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+----------+
|  float64 |             float64 |              float64 | array<int32> | array<float64> |  float64 |    int32 |        float64 |  float64 | array<int32> | str     |             float64 |     bool |
+----------+---------------------+----------------------+--------------+----------------+----------+----------+----------------+----------+--------------+---------+---------------------+----------+
| 5.91e+01 |            0.00e+00 |                   NA | [2]          | [1.00e+00]     | 6.00e+01 |        0 |       1.16e+00 | 2.81e+01 | NA           | NA      |            1.50e+00 |    false |
| 4.07e+01 |            3.44e+00 |                   NA | [2]          | [1.00e+00]     | 5.78e+01 |        0 |             NA | 3.32e+01 | NA           | NA      |                  NA |    false |
| 3.69e+01 |            1.83e+00 |                   NA | [2]          | [1.00e+00]     | 5.97e+01 |        0 |             NA | 2.87e+01 | NA           | NA      |                  NA |    false |
| 4.69e+01 |            2.26e+00 |                   NA | [2]          | [1.00e+00]     | 6.00e+01 |        0 |             NA | 2.96e+01 | NA           | NA      |                  NA |    false |
| 5.64

In [24]:
tbl.export(os.path.join(data_dir, 'filtered_variants.csv'), delimiter=',')

2021-03-02 04:54:09 Hail: INFO: merging 8 files totalling 1.0G...
2021-03-02 04:54:10 Hail: INFO: while writing:
    /home/mschachter/data/simons/filtered_variants.csv
  merge time: 1.279s


# Merging Variants with ClinVar

In [17]:
import pandas as pd
rsid_file = os.path.join(data_dir, 'snps.txt')
df_rsid = pd.read_csv(rsid_file)
df_rsid['rsid'] = df_rsid['rsid'].str.slice(start=2)
i = df_rsid['rsid'] != ''
df_rsid = df_rsid[i]
df_rsid['rsid'] = df_rsid['rsid'].astype('int')

In [18]:
df_rsid.head()

,rsid
0,74512038
1,3131980
2,3094315
3,78408995
4,3131965


In [9]:
clinvar_file = '/home/mschachter/data/clinvar/variant_summary.txt'
df_clinvar = pd.read_csv(clinvar_file, sep='\t')

/home/mschachter/anaconda3/envs/hail/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_clinvar.keys()

Index(['#AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol', 'HGNC_ID',
       'ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated', 'RS# (dbSNP)',
       'nsv/esv (dbVar)', 'RCVaccession', 'PhenotypeIDS', 'PhenotypeList',
       'Origin', 'OriginSimple', 'Assembly', 'ChromosomeAccession',
       'Chromosome', 'Start', 'Stop', 'ReferenceAllele', 'AlternateAllele',
       'Cytogenetic', 'ReviewStatus', 'NumberSubmitters', 'Guidelines',
       'TestedInGTR', 'OtherIDs', 'SubmitterCategories', 'VariationID',
       'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF'],
      dtype='object')

In [11]:
df_clinvar['RS# (dbSNP)'].head()

0    397704705
1    397704705
2    397704709
3    397704709
4    150829393
Name: RS# (dbSNP), dtype: int64

In [26]:
i = (df_clinvar['ClinSigSimple'] == 1) & (df_clinvar['RS# (dbSNP)'].isin(df_rsid['rsid'])) & \
    (df_clinvar['ClinicalSignificance'] == 'Pathogenic')
print('Found {} pathogenic variants'.format(i.sum()))

Found 24 pathogenic variants


In [27]:
df_clinvar[i]

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
1556,15864,single nucleotide variant,NM_000038.6(APC):c.1458T>G (p.Tyr486Ter),324,APC,HGNC:583,Pathogenic,1,"Mar 05, 2019",2229992,...,"criteria provided, single submitter",2,"ACMG2013,ACMG2016",N,OMIM:611731.0032,3,825,112162854,T,G
1557,15864,single nucleotide variant,NM_000038.6(APC):c.1458T>G (p.Tyr486Ter),324,APC,HGNC:583,Pathogenic,1,"Mar 05, 2019",2229992,...,"criteria provided, single submitter",2,"ACMG2013,ACMG2016",N,OMIM:611731.0032,3,825,112827157,T,G
3743,17060,single nucleotide variant,NM_001003841.3(SLC6A19):c.1017-4G>A,340024,SLC6A19,HGNC:27960,Pathogenic,1,"Dec 01, 2008",35329108,...,no assertion criteria provided,1,-,N,OMIM:608893.0005,1,2021,1216900,G,A
3744,17060,single nucleotide variant,NM_001003841.3(SLC6A19):c.1017-4G>A,340024,SLC6A19,HGNC:27960,Pathogenic,1,"Dec 01, 2008",35329108,...,no assertion criteria provided,1,-,N,OMIM:608893.0005,1,2021,1216785,G,A
3819,17101,single nucleotide variant,NM_001002255.2(SUMO4):c.163G>A (p.Val55Met),387082,SUMO4,HGNC:21181,Pathogenic,1,"Feb 01, 2005",237025,...,no assertion criteria provided,1,-,N,"ClinGen:CA115316,OMIM:608829.0001",1,2062,149721690,G,A
3820,17101,single nucleotide variant,NM_001002255.2(SUMO4):c.163G>A (p.Val55Met),387082,SUMO4,HGNC:21181,Pathogenic,1,"Feb 01, 2005",237025,...,no assertion criteria provided,1,-,N,"ClinGen:CA115316,OMIM:608829.0001",1,2062,149400554,G,A
15915,23729,single nucleotide variant,NM_002843.4(PTPRJ):c.827A>C (p.Gln276Pro),5795,PTPRJ,HGNC:9673,Pathogenic,1,"Jul 01, 2002",1566734,...,no assertion criteria provided,1,-,N,"ClinGen:CA250563,UniProtKB:Q12913#VAR_015906,O...",1,8690,48145375,A,C
15916,23729,single nucleotide variant,NM_002843.4(PTPRJ):c.827A>C (p.Gln276Pro),5795,PTPRJ,HGNC:9673,Pathogenic,1,"Jul 01, 2002",1566734,...,no assertion criteria provided,1,-,N,"ClinGen:CA250563,UniProtKB:Q12913#VAR_015906,O...",1,8690,48123823,A,C
22046,27194,single nucleotide variant,NM_000500.9(CYP21A2):c.293-13C>G,1589,CYP21A2,HGNC:2600,Pathogenic,1,"Aug 05, 2020",6467,...,"criteria provided, multiple submitters, no con...",12,-,N,"ClinGen:CA341185,OMIM:613815.0006",3,12155,32006858,C,G
22047,27194,single nucleotide variant,NM_000500.9(CYP21A2):c.293-13C>G,1589,CYP21A2,HGNC:2600,Pathogenic,1,"Aug 05, 2020",6467,...,"criteria provided, multiple submitters, no con...",12,-,N,"ClinGen:CA341185,OMIM:613815.0006",3,12155,32039081,C,G
